# ** Описание **

In [30]:
from __future__ import division
import base64
import csv
import gzip
import zlib

from collections import namedtuple

%matplotlib inline
import matplotlib.pyplot as plt

In [31]:
TRACE_NUM = 1000
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

def trace(items_num, trace_num=TRACE_NUM):
    if items_num % trace_num == 0: logging.info("Complete items %05d" % items_num)

### Утилиты

#### Декораторы

In [32]:
def to_utf8(text):
    if isinstance(text, unicode): text = text.encode('utf8')
    return text

def convert2unicode(f):
    def tmp(text):
        if not isinstance(text, unicode): text = text.decode('utf8')
        return f(text)
    return tmp

def convert2lower(f):
    def tmp(text):        
        return f(text.lower())
    return tmp

#P.S. Декораторы могут усложнять отладку, так что от них вполне можно отказаться и воспользоваться copy-paste

### Извлечение текста из html

#### Извлечение текста при помощи встроенных модулей

In [33]:
from HTMLParser import HTMLParser
import re

###Извлечение текста из title можно вписать сюда

class TextHTMLParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self._text = []
        self._title = ""
        self._in_title = False

    def handle_data(self, data):
        text = data.strip()
        if len(text) > 0:
            text = re.sub('[ \t\r\n]+', ' ', text)
            self._text.append(text + ' ')

    def handle_starttag(self, tag, attrs):
        if tag == 'p':
            self._text.append('\n\n')
        elif tag == 'br':
            self._text.append('\n')
        elif tag == 'title':
            self._in_title = True

    def handle_startendtag(self, tag, attrs):
        if tag == 'br':
            self._text.append('\n\n')

    def text(self):
        return ''.join(self._text).strip()

@convert2unicode
def html2text_parser(text):
    parser = TextHTMLParser()
    parser.feed(text)
    return parser.text()

#### Извлечение текста при помощи дополнительных библиотек

In [34]:
def html2text_bs(raw_html):
    from bs4 import BeautifulSoup
    """
    Тут производится извлечения из html текста
    """
    soup = BeautifulSoup(raw_html, "html.parser")
    [s.extract() for s in soup(['script', 'style'])]
    return soup.get_text()

def html2text_bs_visible(raw_html):
    from bs4 import BeautifulSoup
    """
    Тут производится извлечения из html текста, который видим пользователю
    """
    soup = BeautifulSoup(raw_html, "html.parser")    
    [s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
    return soup.get_text()

def html2text_boilerpipe(raw_html):
    import boilerpipe
    """
    еще одна библиотека очень хорошо извлекающая именно видимый пользователю текст,
    но она завязана на java
    """
    pass

#### Выбираем какой метод для конвертации html в текст будет основным

In [35]:
#html2text = html2text_bs
html2text = html2text_parser

#### Методы для токенизации текста

In [36]:
@convert2lower
@convert2unicode
def easy_tokenizer(text):
    word = unicode()
    for symbol in text:
        if symbol.isalnum(): word += symbol
        elif word:
            yield word
            word = unicode()
    if word: yield word

PYMORPHY_CACHE = {}
MORPH = None
#hint, чтобы установка pymorphy2 не была бы обязательной
def get_lemmatizer():
    import pymorphy2
    global MORPH
    if MORPH is None: MORPH = pymorphy2.MorphAnalyzer()
    return MORPH

@convert2lower
@convert2unicode
def pymorphy_tokenizer(text):
    global PYMORPHY_CACHE
    for word in easy_tokenizer(text):
        word_hash = hash(word)
        if word_hash not in PYMORPHY_CACHE:
            PYMORPHY_CACHE[word_hash] = get_lemmatizer().parse(word)[0].normal_form            
        yield PYMORPHY_CACHE[word_hash]

#### Основная функция, которая вызывается для преобразования html в список слов

In [37]:
def html2word(raw_html, to_text=html2text, tokenizer=easy_tokenizer):
    return to_text(raw_html).lower()

#### Рассчет финальных метрик

In [38]:
def safe_divide(a, b):
    if a == 0: return 0.0
    elif b == 0: return 0.0
    else: return a/b

def calculate_metrics(docs, predictions, threshold):    
    """
    Функция подсчета метрик
    Параметры
    predictions - ранки по документам
    threshold  - порог для метрик
    """
    true_positive = 0
    false_positive = 0
    true_negative = 0
    false_negative = 0
    for i, (url_id, mark, url, prediction) in enumerate(docs):
        mark_predict = predictions[i][1] > threshold

        if mark_predict:                     
            if mark_predict == mark: true_positive += 1
            else: false_positive += 1                    
        else:                     
            if  mark_predict == mark: true_negative += 1
            else: false_negative += 1

    class_prec  = safe_divide(true_positive, true_positive + false_positive)
    class_recall = safe_divide(true_positive, true_positive + false_negative)
        
    class_F1 = safe_divide(2 * class_prec * class_recall, class_prec + class_recall)
    
    
    not_class_prec = safe_divide(true_negative, true_negative + false_negative)
    not_class_recall = safe_divide(true_negative, true_negative + false_positive)
    
    not_class_F1 = safe_divide(2 * not_class_prec * not_class_recall, not_class_prec + not_class_recall)
    
    return ( (class_prec, class_recall, class_F1), (not_class_prec, not_class_recall, not_class_F1) )

def arange(start, stop, step):
    cur_value = start
    while True:
        if cur_value > stop: break
        yield cur_value
        cur_value += step

def plot_results(docs, min_threshold=-1, max_threshold=1, step=0.1, trace=False):
    x = []
    y_p = []
    y_n = []
    docs_predictions = classifier.predict_all(docs)
    for threshold in arange(min_threshold, max_threshold, step):
        r = calculate_metrics(docs, docs_predictions, threshold)
        x.append(threshold)
        y_p.append(r[0])
        y_n.append(r[1])        
        if trace: 
            print 'threshold %s' % threshold
            print '\tclass_prec %s, class_recall %s, class_F1 %s' % r[0]
            print '\tnot_class_prec %s, not_class_recall %s, not_class_F1 %s' % r[1]
            print '\t\tMacroF1Mesure %s' % ((r[0][2] + r[1][2])/2)
    plot_stats(x, y_p, "Class Result")
    plot_stats(x, y_n, "Not class Result")    


def plot_stats(x, y, title):
    plt.figure(figsize=(10, 5))

    prec, = plt.plot( x, 
                     [k[0] for k in y], "r", label='Precision', 
                     linewidth=1)
    accur, = plt.plot( x, 
                      [k[1] for k in y], "b", label='Recall',
                      linewidth=1)
    f1, =    plt.plot( x, 
                      [k[2] for k in y], "g", label='F1',
                      linewidth=1)
    plt.grid(True)
    plt.legend(handles=[prec, accur, f1])
    plt.title(title)
    plt.show()

In [39]:
import json
import sys

def calc_features(url, html_data):
    ws = html2text(html_data).lower()
    words = list(easy_tokenizer(ws))
    words_num = len(words)
    avg_word_len = sum(map(len, words)) * 1.0
    if len(words) > 0:
        avg_word_len /= len(words)
    
    data = json.dumps(''.join(words))
    compression_level = sys.getsizeof(zlib.compress(data)) * 1.0 / sys.getsizeof(data)
    
    title_words_num = 1
    anchor_words_num = 1
    
    return [words_num, avg_word_len, compression_level]

In [40]:
test_html_data = u'''
<html>
<title> Заголовок Ololo </title>
спам 1 2 3
</html>
'''
test_url = 'http://ololo'
test_features = calc_features(test_url, test_html_data)
print test_features

[6, 3.5, 0.68]


In [41]:
DocItem = namedtuple('DocItem', ['doc_id', 'is_spam', 'url', 'features'])

def load_csv(input_file_name, calc_features_f):    
    """
    Загружаем данные и извлекаем на лету признаки
    Сам контент не сохраняется, чтобы уменьшить потребление памяти - чтобы
    можно было запускать даже на ноутбуках в классе
    """
    
    with gzip.open(input_file_name) if input_file_name.endswith('gz') else open(input_file_name)  as input_file:            
        headers = input_file.readline()
        
        for i, line in enumerate(input_file):
            trace(i)
            parts = line.strip().split('\t')
            url_id = int(parts[0])                                        
            mark = bool(int(parts[1]))                    
            url = parts[2]
            pageInb64 = parts[3]
            html_data = base64.b64decode(pageInb64)
            #features = calc_features_f(url, html_data)            
            features = html_data
            yield DocItem(url_id, mark, url, features)
                
        trace(i, 1)     
    

** Обрабатываем входной файл **
<br>
Формат - поля разделенные табуляциями
<br>
0 - идентификатор документа
<br>
1 - метка класса 0 - не спам, 1 - спам
<br>
2 - урл документа
<br>
3 - документ в кодировке base64

Выходной формат - массив кортежей вида
(doc_id, is_spam, url, html_data)

In [42]:
TRAIN_DATA_FILE  = 'data/kaggle_train_data_tab.csv.gz'
# TRAIN_DATA_FILE  = 'kaggle/kaggle_train_data_tab_300.csv.gz'

train_docs = list(load_csv(TRAIN_DATA_FILE, calc_features))

23:58:53 INFO:Complete items 00000
23:58:54 INFO:Complete items 01000
23:58:55 INFO:Complete items 02000
23:58:56 INFO:Complete items 03000
23:58:57 INFO:Complete items 04000
23:58:58 INFO:Complete items 05000
23:58:59 INFO:Complete items 06000
23:59:00 INFO:Complete items 07000
23:59:00 INFO:Complete items 07043


** Классификатор: **
Нужно реализовать

In [49]:
import numpy as np

from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier

#taken from 
#http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
class ClassifierSGD:    
    def __init__(self):
        self.clf = Pipeline([
                              ('vect', CountVectorizer()),
                              ('tfidf', TfidfTransformer()),
                              ('clf', SGDClassifier())
                            ])

    def features(self, doc):        
        return doc.features
    
    def predict_all(self, docs):
        fdocs = map(self.features, docs)
        predictions = self.clf.predict(fdocs)
        return zip(map(lambda doc: doc.doc_id, docs), predictions)
    
    def train(self, docs):
        X = map(lambda doc: doc.features, docs)
        Y = map(lambda doc: 1 if doc.is_spam else 0, docs)
        self.clf.fit(X, Y)
        

In [50]:
classifier = ClassifierSGD()
classifier.train(train_docs)


** Рисуем графики **

In [55]:
#plot_results(train_docs, min_threshold=0, max_threshold=3000, step=100, trace=1)


In [52]:
TEST_DATA_FILE  = 'data/kaggle_test_data_tab.csv.gz'
# TEST_DATA_FILE  = 'kaggle/kaggle_train_data_tab_300.csv.gz'

test_docs = list(load_csv(TEST_DATA_FILE, calc_features))

23:59:51 INFO:Complete items 00000
23:59:52 INFO:Complete items 01000
23:59:53 INFO:Complete items 02000
23:59:54 INFO:Complete items 03000
23:59:55 INFO:Complete items 04000
23:59:55 INFO:Complete items 05000
23:59:56 INFO:Complete items 06000
23:59:57 INFO:Complete items 07000
23:59:59 INFO:Complete items 08000
00:00:02 INFO:Complete items 09000
00:00:05 INFO:Complete items 10000
00:00:07 INFO:Complete items 11000
00:00:08 INFO:Complete items 12000
00:00:10 INFO:Complete items 13000
00:00:11 INFO:Complete items 14000
00:00:13 INFO:Complete items 15000
00:00:15 INFO:Complete items 16000
00:00:15 INFO:Complete items 16038


In [53]:
with open('my_submission-new.csv', 'wb') as fout:
    writer = csv.writer(fout)
    writer.writerow(['Id','Prediction'])
    predictions = classifier.predict_all(test_docs)
    for item in predictions:
        writer.writerow([item[0], item[1]])
    print('Answer is ready')

Answer is ready
